# Практическая работа №1

## Извлечение признаков из данных

### Чтение информации о матчах

In [3]:
import json
import bz2

with bz2.BZ2File('./data/matches.jsonlines.bz2') as matches_file:
    for line in matches_file:
        match = json.loads(line)

        # Обработка матча
        brea

### Пример чтения файла с признаками

In [4]:
import pandas
features = pandas.read_csv('./data/features.csv', index_col='match_id')

features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


## Реализация гардиентного бустинга в лоб
1. Считывание данных из файла и удаление признаков с результатами матча

In [9]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

# 1. Считываем таблицу с признаками из файла features.csv
data = pd.read_csv('features.csv')

# Удаляем признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке)
features_to_drop = ['duration', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire']
data = data.drop(features_to_drop, axis=1)

2. Проверка на наличие пропусков

In [10]:
# 2. Проверяем выборку на наличие пропусков
missing_values = data.isnull().sum()
print("Признаки с пропусками:")
print(missing_values[missing_values > 0])

# В данном случае мы заменим пропуски на нули.
data = data.fillna(0)

Признаки с пропусками:
first_blood_time               19553
first_blood_team               19553
first_blood_player1            19553
first_blood_player2            43987
radiant_bottle_time            15691
radiant_courier_time             692
radiant_flying_courier_time    27479
radiant_first_ward_time         1836
dire_bottle_time               16143
dire_courier_time                676
dire_flying_courier_time       26098
dire_first_ward_time            1826
dtype: int64


3. Поиск целевой переменной

In [11]:
# 3. Найдем название столбца с целевой переменной. В данной задаче это 'radiant_win'.
target_column = 'radiant_win'

4. Подготовка данных к кросс-валидации, а так же генератор разбиений

In [12]:
# 4. Подготовим данные для обучения и кросс-валидации
X = data.drop(target_column, axis=1)
y = data[target_column]

# Зафиксируем генератор разбиений для кросс-валидации
kf = KFold(n_splits=5, shuffle=True, random_state=42)

5. Оценка качества гардиентного бустинга

In [13]:
# 5. Оценим качество градиентного бустинга с помощью кросс-валидации
n_estimators_values = [10, 20, 30]  # Разные значения для количества деревьев
results = {}

for n_estimators in n_estimators_values:
    clf = GradientBoostingClassifier(n_estimators=n_estimators, random_state=42)
    scores = cross_val_score(clf, X, y, cv=kf, scoring='roc_auc')
    results[n_estimators] = np.mean(scores)

6. Вывод результатов

In [14]:
# Выведем результаты
for n_estimators, score in results.items():
    print(f"Количество деревьев: {n_estimators}, Средний ROC AUC: {score}")

Количество деревьев: 10, Средний ROC AUC: 0.6648506879750012
Количество деревьев: 20, Средний ROC AUC: 0.6824618768044435
Количество деревьев: 30, Средний ROC AUC: 0.6900064710388155
